In [1]:
from pgmpy.models import BayesianModel
from pgmpy.factors.discrete import TabularCPD

### Model and CPDs

<img src="Traffic Prioritisation BayesNet.png" />

### Generating the network

In [34]:
def generate_network_and_cpds():
    # Defining the model structure. We can define the network by just passing a list of edges.
    model = BayesianModel([('W', 'P'), ('P', 'B'), ('P', 'N'), ('N', 'C'),
                       ('B', 'C'), ('B', 'R'), ('C', 'S'), ('R', 'S'),
                       ('SP', 'S')])
    cpd_w_sn = TabularCPD(variable='W', variable_card=2, values=[[0.7], [0.3]], state_names={'W': ['Mon-Fri', 'Sat-Sun']})

    cpd_p_sn = TabularCPD(variable='P', variable_card=4, 
                          values=[[0.125, 0],
                                  [0.25, 0.5],
                                  [0.125, 0],
                                  [0.5, 0.5]],
                          evidence=['W'],
                          evidence_card=[2],
                          state_names={'W': ['Mon-Fri', 'Sat-Sun'],
                                       'P': ['P1(7-10AM)', 'P2(10AM-4PM)', 'P3(4PM-7PM)', 'P4(7PM-7AM)']})

    cpd_n_sn = TabularCPD(variable='N', variable_card=2, 
                          values=[[0.9, 0.1, 0.9, 0.1],
                                  [0.1, 0.9, 0.1, 0.9]],
                          evidence=['P'],
                          evidence_card=[4],
                          state_names={'N': ['N0', 'N1'],
                                       'P': ['P1(7-10AM)', 'P2(10AM-4PM)', 'P3(4PM-7PM)', 'P4(7PM-7AM)']})

    cpd_b_sn = TabularCPD(variable='B', variable_card=2, 
                          values=[[1, 0, 1, 0],
                                  [0, 1, 0, 1]],
                          evidence=['P'],
                          evidence_card=[4],
                          state_names={'P': ['P1(7-10AM)', 'P2(10AM-4PM)', 'P3(4PM-7PM)', 'P4(7PM-7AM)'],
                                       'B': ['Busy', 'Not Busy']})

    cpd_c_sn = TabularCPD(variable='C', variable_card=2, 
                          values=[[0.05, 0.6, 0.4, 0.95],
                                  [0.95, 0.4, 0.6, 0.05]],
                          evidence=['N', 'B'],
                          evidence_card=[2, 2],
                          state_names={'C': ['Low', 'High'],
                                       'N': ['N0', 'N1'],
                                       'B': ['Busy', 'Not Busy']})

    cpd_r_sn = TabularCPD(variable='R', variable_card=2, 
                          values=[[0.005, 0.015],
                                  [0.995, 0.985]],
                          evidence=['B'],
                          evidence_card=[2],
                          state_names={'R': ['Roadwork', 'No Roadwork'],
                                       'B': ['Busy', 'Not Busy']})

    cpd_sp_sn = TabularCPD(variable='SP', variable_card=5, 
                          values=[[0.65], [0.117], [0.117], [0.058], [0.058]],
                          state_names={'SP': ['SP_E', 'SP_DCBA', 'SP_CBAD', 'SP_BADC', 'SP_ADCB']})

    cpd_s_sn = TabularCPD(variable='S', variable_card=5, 
                          values=[[0.9, 0.9, 0.117, 0.75, 0.9, 0.9, 0.117, 0.75, 0.9, 0.9, 0.058, 0.75, 0.9, 0.9, 0.058, 0.75, 0.9, 0.9, 0.65, 0.75],
                                  [0.025, 0.025, 0.65, 0.0625, 0.025, 0.025, 0.117, 0.0625, 0.025, 0.025, 0.117, 0.0625, 0.025, 0.025, 0.117, 0.0625, 0.025, 0.025, 0.117, 0.0625],
                                 [0.025, 0.025, 0.117, 0.0625, 0.025, 0.025, 0.65, 0.0625, 0.025, 0.025, 0.117, 0.0625, 0.025, 0.025, 0.117, 0.0625, 0.025, 0.025, 0.117, 0.0625],
                                 [0.025, 0.025, 0.058, 0.0625, 0.025, 0.025, 0.058, 0.0625, 0.025, 0.025, 0.65, 0.0625, 0.025, 0.025, 0.058, 0.0625, 0.025, 0.025, 0.058, 0.0625],
                                 [0.025, 0.025, 0.058, 0.0625, 0.025, 0.025, 0.058, 0.0625, 0.025, 0.025, 0.058, 0.0625, 0.025, 0.025, 0.65, 0.0625, 0.025, 0.025, 0.058, 0.0625]],
                          evidence=['SP', 'R', 'C'],
                          evidence_card=[5, 2, 2],
                          state_names={'S': ['SP_E', 'SP_DCBA', 'SP_CBAD', 'SP_BADC', 'SP_ADCB'],
                              'SP': ['SP_E', 'SP_DCBA', 'SP_CBAD', 'SP_BADC', 'SP_ADCB'],
                                       'R': ['Roadwork', 'No Roadwork'],
                                       'C': ['Low', 'High']})

    model.add_cpds(cpd_w_sn, cpd_p_sn, cpd_n_sn, cpd_b_sn, cpd_c_sn, cpd_r_sn, cpd_sp_sn, cpd_s_sn)
    print('Model state: ', model.check_model())
    if model.check_model() == True:
        return model

In [35]:
model = generate_network_and_cpds()

Model state:  True


### Showing the conditional tables

In [36]:
def print_cpds(model):
    print('Weekdays Table')
    print(model.get_cpds('W'))
    print('Periods Table')
    print(model.get_cpds('P'))
    print('Commutes Table')
    print(model.get_cpds('N'))
    print('Busy Table')
    print(model.get_cpds('B'))
    print('Congestion Table')
    print(model.get_cpds('C'))
    print('Roadworks Table')
    print(model.get_cpds('R'))
    print('Previous Sequence')
    print(model.get_cpds('SP'))
    print('Output Sequence')
    print(model.get_cpds('S'))

In [37]:
print_cpds(model)

Weekdays Table
╒════════════╤═════╕
│ W(Mon-Fri) │ 0.7 │
├────────────┼─────┤
│ W(Sat-Sun) │ 0.3 │
╘════════════╧═════╛
Periods Table
╒═════════════════╤════════════╤════════════╕
│ W               │ W(Mon-Fri) │ W(Sat-Sun) │
├─────────────────┼────────────┼────────────┤
│ P(P1(7-10AM))   │ 0.125      │ 0.0        │
├─────────────────┼────────────┼────────────┤
│ P(P2(10AM-4PM)) │ 0.25       │ 0.5        │
├─────────────────┼────────────┼────────────┤
│ P(P3(4PM-7PM))  │ 0.125      │ 0.0        │
├─────────────────┼────────────┼────────────┤
│ P(P4(7PM-7AM))  │ 0.5        │ 0.5        │
╘═════════════════╧════════════╧════════════╛
Commutes Table
╒═══════╤═══════════════╤═════════════════╤════════════════╤════════════════╕
│ P     │ P(P1(7-10AM)) │ P(P2(10AM-4PM)) │ P(P3(4PM-7PM)) │ P(P4(7PM-7AM)) │
├───────┼───────────────┼─────────────────┼────────────────┼────────────────┤
│ N(N0) │ 0.9           │ 0.1             │ 0.9            │ 0.1            │
├───────┼───────────────┼────────

In [38]:
model.get_cpds()

[<TabularCPD representing P(W:2) at 0x1c22292dcf8>,
 <TabularCPD representing P(P:4 | W:2) at 0x1c22292d898>,
 <TabularCPD representing P(N:2 | P:4) at 0x1c22292d048>,
 <TabularCPD representing P(B:2 | P:4) at 0x1c22292d0f0>,
 <TabularCPD representing P(C:2 | N:2, B:2) at 0x1c22292da20>,
 <TabularCPD representing P(R:2 | B:2) at 0x1c22292d9e8>,
 <TabularCPD representing P(SP:5) at 0x1c22292d278>,
 <TabularCPD representing P(S:5 | SP:5, R:2, C:2) at 0x1c22292db00>]

In [39]:
model.get_cardinality('S')

5

In [40]:
# Getting all the local independencies in the network.
model.local_independencies(['W', 'P', 'N', 'B', 'C', 'R', 'SP', 'S'])

(W _|_ SP)
(P _|_ SP | W)
(N _|_ B, W, SP, R | P)
(B _|_ N, SP, W | P)
(C _|_ P, W, SP, R | N, B)
(R _|_ N, SP, W, C, P | B)
(SP _|_ N, B, P, W, C, R)
(S _|_ N, B, W, P | C, SP, R)

In [41]:
print('Active trail nodes for the output sequence: ', model.active_trail_nodes('S'))

Active trail nodes for the output sequence:  {'S': {'S', 'B', 'N', 'C', 'R', 'P', 'SP', 'W'}}


### Inference

#### Query for S given all the data

In [49]:
from pgmpy.inference import VariableElimination
def var_elimination_inference(model, query_vars, index, evidence={}):   
    infer = VariableElimination(model)
    s_dist = infer.query(query_vars, evidence=evidence)[index]
    print('S_0: E, S_1: DCBA, S_2: CBAD, S_3: BADC, S_4: ADCB')
    print(s_dist)

In [50]:
var_elimination_inference(model, ['S'], 'S')

S_0: E, S_1: DCBA, S_2: CBAD, S_3: BADC, S_4: ADCB
╒═════╤══════════╕
│ S   │   phi(S) │
╞═════╪══════════╡
│ S_0 │   0.2876 │
├─────┼──────────┤
│ S_1 │   0.3661 │
├─────┼──────────┤
│ S_2 │   0.1506 │
├─────┼──────────┤
│ S_3 │   0.1111 │
├─────┼──────────┤
│ S_4 │   0.0846 │
╘═════╧══════════╛


#### 1. $ P(S | C_0, R_1, SP_E) $ - probability of each sequence given low congestion, no roadworks, and  previous sequence E.

In [51]:
var_elimination_inference(model, ['S'], 'S', evidence={'C': 0, 'R': 1, 'SP': 0})

S_0: E, S_1: DCBA, S_2: CBAD, S_3: BADC, S_4: ADCB
╒═════╤══════════╕
│ S   │   phi(S) │
╞═════╪══════════╡
│ S_0 │   0.1170 │
├─────┼──────────┤
│ S_1 │   0.6500 │
├─────┼──────────┤
│ S_2 │   0.1170 │
├─────┼──────────┤
│ S_3 │   0.0580 │
├─────┼──────────┤
│ S_4 │   0.0580 │
╘═════╧══════════╛


#### 2. $ P(S | C_1, R_1, SP_{DCBA}) $ - probability of each sequence given high congestion, no roadworks, and  previous sequence DCBA.

In [52]:
var_elimination_inference(model, ['S'], 'S', evidence={'C': 1, 'R': 1, 'SP': 1})

S_0: E, S_1: DCBA, S_2: CBAD, S_3: BADC, S_4: ADCB
╒═════╤══════════╕
│ S   │   phi(S) │
╞═════╪══════════╡
│ S_0 │   0.7500 │
├─────┼──────────┤
│ S_1 │   0.0625 │
├─────┼──────────┤
│ S_2 │   0.0625 │
├─────┼──────────┤
│ S_3 │   0.0625 │
├─────┼──────────┤
│ S_4 │   0.0625 │
╘═════╧══════════╛


#### 3. $ P(S | C_0, R_0, SP_{BADC}) $ - probability of each sequence given low congestion, roadworks, and  previous sequence BADC.

In [53]:
var_elimination_inference(model, ['S'], 'S', evidence={'C': 0, 'R': 0, 'SP': 3})

S_0: E, S_1: DCBA, S_2: CBAD, S_3: BADC, S_4: ADCB
╒═════╤══════════╕
│ S   │   phi(S) │
╞═════╪══════════╡
│ S_0 │   0.9000 │
├─────┼──────────┤
│ S_1 │   0.0250 │
├─────┼──────────┤
│ S_2 │   0.0250 │
├─────┼──────────┤
│ S_3 │   0.0250 │
├─────┼──────────┤
│ S_4 │   0.0250 │
╘═════╧══════════╛


#### 4. $ P(S | C_1, R_0, SP_{ADCB}) $ - probability of each sequence given high congestion, roadworks, and  previous sequence ADCB.

In [54]:
var_elimination_inference(model, ['S'], 'S', evidence={'C': 1, 'R': 0, 'SP': 4})

S_0: E, S_1: DCBA, S_2: CBAD, S_3: BADC, S_4: ADCB
╒═════╤══════════╕
│ S   │   phi(S) │
╞═════╪══════════╡
│ S_0 │   0.9000 │
├─────┼──────────┤
│ S_1 │   0.0250 │
├─────┼──────────┤
│ S_2 │   0.0250 │
├─────┼──────────┤
│ S_3 │   0.0250 │
├─────┼──────────┤
│ S_4 │   0.0250 │
╘═════╧══════════╛


#### 5. $ P(S | C_0, R_1, SP_{BADC}) $ - probability of each sequence given low congestion, no roadworks, and previous sequence BADC.

In [55]:
var_elimination_inference(model, ['S'], 'S', evidence={'C': 0, 'R': 1, 'SP': 3})

S_0: E, S_1: DCBA, S_2: CBAD, S_3: BADC, S_4: ADCB
╒═════╤══════════╕
│ S   │   phi(S) │
╞═════╪══════════╡
│ S_0 │   0.0580 │
├─────┼──────────┤
│ S_1 │   0.1170 │
├─────┼──────────┤
│ S_2 │   0.1170 │
├─────┼──────────┤
│ S_3 │   0.0580 │
├─────┼──────────┤
│ S_4 │   0.6500 │
╘═════╧══════════╛


### Making the predictions

#### Output sequence given all the data

In [22]:
seq_dict = {0: 'E', 1: 'DCBA', 2: 'CBAD', 3: 'BACD', 4: 'ACDB'}

In [59]:
def var_elimination_predict(model, query_vars, evidence={}):
    infer = VariableElimination(model)
    predictions = infer.map_query(query_vars, evidence=evidence)
    print({k: seq_dict.get(v, v) for k, v in predictions.items()})

In [60]:
var_elimination_predict(model, ['S'])

{'S': 'DCBA'}


#### Output sequence given weekdays, a larger number of commutes and P1 (busy period):
$ P(S | W_0, N_0, P_0) $

In [61]:
var_elimination_predict(model, ['S'], evidence={'W': 0, 'N': 0, 'P': 0})

{'S': 'E'}


#### Output sequence given weekdays, P3 (busy period) and high congestion:
$ P(S | W_0, P_2, C_1) $

In [63]:
var_elimination_predict(model, ['S'], evidence={'W': 0, 'C': 1, 'P': 2})

{'S': 'E'}


#### Output sequence given weekdays, P3 (busy period) and low congestion:
$ P(S | W_0, P_2, C_0) $

In [64]:
var_elimination_predict(model, ['S'], evidence={'W': 0, 'C': 0, 'P': 2})

{'S': 'DCBA'}


#### Output sequence given weekdays, P3 (busy period), low congestion and previous sequence DCBA:

$ P(S | W_0, C_0, P_2, SP_{DCBA}) $

In [65]:
var_elimination_predict(model, ['S'], evidence={'W': 0, 'C': 0, 'P': 2, 'SP': 1})

{'S': 'CBAD'}


#### Output sequence given weekends, P2 (non-busy period), low congestion, no roadworks:

$ P(S | W_1, C_0, R_1, SP_{DCBA}) $

In [66]:
var_elimination_predict(model, ['S'], evidence={'W': 1, 'C': 0, 'P': 1, 'R': 1})

{'S': 'DCBA'}


### Likelihood sampling

In [67]:
from pgmpy.sampling import BayesianModelSampling
from pgmpy.factors.discrete import State

In [72]:
inference = BayesianModelSampling(model)
evidence = [State('C', 1), State('R', 1)]
inference.likelihood_weighted_sample(evidence=evidence, size=50, return_type='dataframe')

,SP,W,P,B,R,N,C,S,_weight
0,1,1,3,1,1,1,1,0,0.04925
1,0,1,3,1,1,0,1,0,0.39400
2,0,1,1,1,1,1,1,0,0.04925
3,0,0,3,1,1,1,1,0,0.04925
4,1,0,3,1,1,1,1,0,0.04925
5,0,0,1,1,1,1,1,0,0.04925
6,0,0,3,1,1,1,1,0,0.04925
7,0,0,3,1,1,1,1,4,0.04925
8,0,0,3,1,1,1,1,0,0.04925
9,2,0,3,1,1,1,1,4,0.04925
